In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head()

In [4]:
movies.head(1)

In [5]:
credits.head()

In [6]:
credits.head()['cast'].values

In [7]:
movies=movies.merge(credits,on='title')

In [8]:
movies.shape

In [9]:
movies['original_language'].value_counts()

In [10]:
movies.head(1)

In [11]:
# genre
# id
# keywords
# title
# Overview
# Cast
# Crew

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [12]:
movies.head()

In [13]:
movies.isnull().sum()

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.duplicated().sum()

In [16]:
movies.iloc[0]

In [17]:
movies.iloc[0].genres

In [18]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [19]:
movies['genres']=movies['genres'].apply(convert)

In [20]:
movies.head()

In [21]:
movies['keywords']=movies['keywords'].apply(convert)

In [22]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if (counter!=3):
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [23]:
movies['cast']=movies['cast'].apply(convert3)

In [24]:
movies.head()

In [25]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=="Director":
            L.append(i['name'])
            break
    return L

In [26]:
movies['crew']=movies['crew'].apply(fetch_director)

In [27]:
movies.head()

In [28]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [29]:
movies.head()

In [30]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [31]:
movies.head()

In [32]:
movies['tag']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [33]:
movies.head()

In [34]:
new_df=movies[['movie_id','title','tag']]

In [35]:
new_df

In [36]:
new_df['tag']=new_df['tag'].apply(lambda x:" ".join(x))

In [37]:
new_df.head()

In [38]:
new_df['tag'][0]

In [39]:
new_df['tag']=new_df['tag'].apply(lambda x:x.lower())

In [40]:
new_df.head()

In [41]:
!pip install nltk

In [42]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [43]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
        

In [44]:
new_df['tag']=new_df['tag'].apply(stem)

In [45]:
new_df['tag'][0]

In [46]:
new_df['tag'][1]

## Text vectorisation - Bag of words

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [48]:
cv.fit_transform(new_df['tag']).toarray()

In [49]:
cv.fit_transform(new_df['tag']).toarray().shape

In [50]:
vectors=cv.fit_transform(new_df['tag']).toarray()

In [51]:
vectors[0]

In [52]:
cv.get_feature_names_out()

In [53]:
# Stemming
#[dances,dancing,dance]=[dance,dance,dance]


In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
cosine_similarity(vectors).shape

In [56]:
similarity=cosine_similarity(vectors)

In [57]:
similarity

In [58]:
similarity[0]

In [59]:
new_df[new_df['title'] == "Batman Begins"].index[0]

In [60]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [61]:
def recommend(movie):
    movie_index=new_df[new_df['title'] == movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        name=new_df.iloc[i[0]].title
        print(name)

In [62]:
recommend("Batman Begins")

In [63]:
new_df.iloc[3297]

In [64]:
import pickle
pickle.dump(new_df,open("movies.pkl","wb"))

In [65]:
pickle.dump(similarity,open('similarity.pkl',"wb"))